In [1]:
from selenium import webdriver
import pandas as pd
import os
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pymysql
import pandas as pd
from sqlalchemy import create_engine
from db import *

In [2]:


wd = webdriver.Chrome()
wd.get('https://stat.molit.go.kr/portal/cate/statMetaView.do?hRsId=58&hFormId=1244&hSelectId=1244&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFormId=1244&sStart=2015&sEnd=2023&sStyleNum=562&settingRadio=xlsx')


element = wd.find_element(By.XPATH, '//*[@id="main"]/div/div[3]/div[1]/div/div[2]/div[2]/div/dl/dd[1]/ul/li[1]/a/em')
wd.execute_script("arguments[0].click();", element)

html = wd.page_source
soup = BeautifulSoup(html,'html.parser')


In [3]:
download_folder = os.path.expanduser('~/Downloads')
file_name = soup.select('#main > div > div.contents-wrap > div.contents > div > div:nth-child(2) > div.mu-accordion-body > div > dl > dd:nth-child(2) > ul > li:nth-child(1) > a > em')[0].text[2:]
file_name = file_name.replace(' ','_')
file_name
file_path = os.path.join(download_folder, file_name)

# .xlsx 파일을 읽어옵니다
try:
    df = pd.read_excel(file_path,sheet_name ='19.연도별 자동차 등록현황',engine ='openpyxl', header = [1,2])
    print("파일이 성공적으로 읽혔습니다.")
    print(df.head())  # 데이터의 첫 5행을 출력
except FileNotFoundError:
    print("지정한 파일이 존재하지 않습니다.")
except Exception as e:
    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")

파일이 성공적으로 읽혔습니다.
  조회년월: 2024.07                                                              \
     차종      승용      승용.1    승용.2      승용.3     승합    승합.1    승합.2     승합.3   
0    년도      관용       자가용     영업용         계     관용     자가용     영업용        계   
1  2007   20714  11674085  404980  12099779  12650  999807   92492  1104949   
2  2008   21388  12025715  436706  12483809  13269  987448   95981  1096698   
3  2009   22267  12551833  449719  13023819  14177  967890   98620  1080687   
4  2010   22872  13124972  483925  13631769  15039  931740  102946  1049725   

          ...                                                               \
      화물  ...    화물.2     화물.3    특수   특수.1   특수.2   특수.3     합계      합계.1   
0     관용  ...     영업용        계    관용    자가용    영업용      계     관용       자가용   
1  25230  ...  334584  3171351  2090  10945  39063  52098  60684  15496374   
2  25535  ...  338711  3160338  2110  11372  39892  53374  62302  15820627   
3  25970  ...  341745  3166512  2070  1

In [10]:
df = df.dropna(how='all',axis=1)
df = df.dropna(how='all',axis=0)
car_data = df.loc[13:]
car_data

조회년월: 2024.07                                                              \
      차종      승용      승용.1     승용.2      승용.3     승합    승합.1    승합.2    승합.3   
13  2019   33344  18071723  1072450  19177517  23014  663494  125291  811799   
14  2020   33619  18686570  1140766  19860955  23938  639504  120400  783842   
15  2021   34548  19168262  1207838  20410648  24460  608894  116614  749968   
16  2022   35071  19647975  1269713  20952759  24755  583625  115581  723961   
17  2023   35481  20079294  1275427  21390202  25009  555119  114446  694574   

           ...                                                                \
       화물  ...    화물.2     화물.3    특수   특수.1   특수.2    특수.3     합계      합계.1   
13  33072  ...  416412  3592586  2867  30043  62554   95464  92297  21908362   
14  33391  ...  425252  3615245  3009  39010  63918  105937  93957  22521686   
15  33800  ...  438331  3631975  3141  49733  65636  118510  95949  22986733   
16  34587  ...  450609  3696317  3441  59623  66977  130041  97854  23502344   
17  35318  ...  465888  3726400  3702  66655  67668  138025  99510  23926262   

                       
       합계.2      합계.3  
13  1676707  23677366  
14  1750336  24365979  
15  1828419  24911101  
16  1902880  25503078  
17  1923429  25949201  

[5 rows x 21 columns]

In [6]:
df_1 =  pd.read_excel(file_path,sheet_name='01.통계표',engine ='openpyxl',header=[3])
df_1 = df_1.loc[1:17]
df_1 = df_1.dropna(axis=1)
df_1.columns = column_names = ['region', '승용 관용', '승용 자가용', '승용 영업용', '승용 총계', '승합 관용', '승합 자가용', '승합 영엉용', '승합 총계', '화물 관용',\
                   '화물 자가용', '화물 영업용', '화물 총계', '특수 관용', '특수 자가용', '특수 영업용', '특수 총계', '합계 관용', '합계 자가용', '합계 영업용', '합계 총계']

df_1[['region']]

,region
1,서울
2,부산
3,대구
4,인천
5,광주
6,대전
7,울산
8,세종
9,경기
10,충북


In [5]:
Insert_data(df_1,'carfirst','car1','region')

OperationalError: (1170, "BLOB/TEXT column 'region' used in key specification without a key length")